In [ ]:
import geomesa_pyspark 
conf = geomesa_pyspark.configure(jars=['/usr/lib/spark/jars/geomesa-hbase-spark-runtime_2.11-2.1.0-m.2.jar'],
                                 packages=['geomesa_pyspark','pytz'], 
                                 spark_home='/usr/lib/spark/') 
#setAppName('MyTestApp')
conf.get('spark.master') # u'yarn'
from pyspark.sql import SparkSession
spark = ( SparkSession.builder.config(conf=conf).getOrCreate() )

params = {    "hbase.zookeepers": "hbase.optix-ons-local:2181",    "hbase.catalog": "ons-historical" } 
feature = "ee" 
ee = ( spark    .read    .format("geomesa")    .options(**params)    .option("geomesa.feature", feature)    .load() )
ee.createOrReplaceTempView("ee")

In [ ]:
df1=spark.sql("""
SELECT
    latitude,
    longitude,
    mmsi,
    vessel_type_code,
    cast(dtg as date)
FROM
    ee
WHERE
    nav_status_code = '5'
and
    vessel_type_code IN (
    '60', '61', '62', '63', '64', '65', '66', '67', '68', '69',
    '70', '71', '72', '73', '74', '75', '76', '77', '78', '79',
    '80', '81', '82', '83', '84', '85', '86', '87', '88', '89',
    '90', '91', '92', '93', '94', '95', '96', '97', '98', '99' 
    )
and
    dtg >= cast(('2020-09-01 00:00:00') as timestamp)
and
    dtg >= cast(('2020-09-07 00:00:00') as timestamp)
order by
    dtg
asc
""").coalesce(1).write.option("header", "true").csv("s3://optix.ons.jupyter/jupyter/dwater/folder/rodrigo/porto_1")

# .coalesce(1).write.option("header", "true").csv("s3://optix.ons.jupyter/jupyter/dwater/folder/forecast_7")

In [ ]:
df1=spark.sql("""
SELECT
    mmsi,
    vessel_type_code,
    dtg,
    message_type
FROM
    ee
WHERE
    message_type IN (13, 14)
and
    dtg >= cast(('2020-01-01 00:00:00') as timestamp)
and
    dtg >= cast(('2020-09-07 00:00:00') as timestamp)
order by
    dtg
asc
""").show(10)